# CO543 - Image Processing
## Lab 04 - Part 2 using ANN

- E/19/129
- K. H. Gunawardana


In [1]:
import os
from google.colab import drive

path = "/content/drive/My Drive/ML/CO543/lab04/"

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [3]:
train_df = pd.read_csv(path + "fashion-mnist_train.csv")
test_df = pd.read_csv(path + "fashion-mnist_test.csv")

In [4]:
train_df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
test_df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,32,23,14,20,0,0,1,0,0,0
9996,6,0,0,0,0,0,0,0,0,0,...,0,0,0,2,52,23,28,0,0,0
9997,8,0,0,0,0,0,0,0,0,0,...,175,172,172,182,199,222,42,0,1,0
9998,8,0,1,3,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [6]:
from sklearn.preprocessing import StandardScaler

X_train = train_df.iloc[:, 1:].values.astype("float32") / 255.0
y_train = train_df["label"].values.astype("int32")
X_test = test_df.iloc[:, 1:].values.astype("float32") / 255.0
y_test = test_df["label"].values.astype("int32")

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [17]:

model = keras.models.Sequential([
    keras.layers.Dense(728, activation="relu", input_shape=(28 * 28,)),  # 728 neurons fully connected layer
    keras.layers.Dense(400, activation="relu"),  # 400 neurons fully connected layer
    keras.layers.Dropout(0.2),
    keras.layers.Dense(200, activation="relu"),  # 200 neurons fully connected layer
    keras.layers.Dense(75, activation="relu"),   # 75 neurons fully connected layer
    keras.layers.Dense(10, activation="softmax") # Output layer with 10 neurons (10 classes) and softmax activation
])


In [18]:
from tensorflow.keras.callbacks import LearningRateScheduler

# Learning rate scheduler (optional)
def scheduler(epoch, lr):
  if epoch < 5:
    return lr
  else:
    return lr * 0.1  # Reduce learning rate after 5 epochs

model.compile(optimizer=Adam(learning_rate=0.001),  # Lower initial learning rate
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Learning rate schedule callback (optional)
learning_rate_scheduler = LearningRateScheduler(scheduler)

In [19]:
history = model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), callbacks=[learning_rate_scheduler])

Epoch 1/15
1500/1500 [==============================] - 23s 14ms/step - loss: 0.5035 - accuracy: 0.8199 - val_loss: 0.3819 - val_accuracy: 0.8665 - lr: 0.0010
Epoch 2/15
1500/1500 [==============================] - 21s 14ms/step - loss: 0.3864 - accuracy: 0.8607 - val_loss: 0.3716 - val_accuracy: 0.8652 - lr: 0.0010
Epoch 3/15
1500/1500 [==============================] - 22s 14ms/step - loss: 0.3502 - accuracy: 0.8733 - val_loss: 0.3496 - val_accuracy: 0.8719 - lr: 0.0010
Epoch 4/15
1500/1500 [==============================] - 21s 14ms/step - loss: 0.3191 - accuracy: 0.8848 - val_loss: 0.3292 - val_accuracy: 0.8800 - lr: 0.0010
Epoch 5/15
1500/1500 [==============================] - 24s 16ms/step - loss: 0.2991 - accuracy: 0.8928 - val_loss: 0.3249 - val_accuracy: 0.8822 - lr: 0.0010
Epoch 6/15
1500/1500 [==============================] - 21s 14ms/step - loss: 0.2212 - accuracy: 0.9174 - val_loss: 0.2820 - val_accuracy: 0.8992 - lr: 1.0000e-04
Epoch 7/15
1500/1500 [====================

In [21]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

313/313 [==============================] - 1s 4ms/step - loss: 0.2806 - accuracy: 0.8998
Test accuracy: 0.8998000025749207
